In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [32]:
import pickle
import pandas as pd
from sklearn import metrics
from numpy import std
from numpy import mean

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('../../data/taxi/yellow_tripdata_2022-02.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [14]:
#Question 1: Standard deviation for February 2022 Yellow dataset
std(y_pred)

5.28140357655334

In [20]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [23]:
df['prediction'] = y_pred.tolist()
df_result = df[['ride_id', 'prediction']]

In [27]:
df_result

,ride_id,prediction
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904
...,...,...
2979426,2022/02_2979426,12.038225
2979427,2022/02_2979427,11.441569
2979428,2022/02_2979428,11.890459
2979429,2022/02_2979429,15.102681


In [28]:
with open('predictions', 'wb') as output_file:
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [31]:
df2 = read_data('../../data/taxi/yellow_tripdata_2022-03.parquet')
dicts2 = df2[categorical].to_dict(orient='records')
X_val2 = dv.transform(dicts)
y_pred2 = model.predict(X_val2)

In [33]:
mean(y_pred2)

12.513422116701408

In [34]:
df3 = read_data('../../data/taxi/yellow_tripdata_2022-04.parquet')
dicts3 = df3[categorical].to_dict(orient='records')
X_val3 = dv.transform(dicts)
y_pred3 = model.predict(X_val3)

In [35]:
mean(y_pred3)

12.513422116701408